In [314]:

%matplotlib widget

import cell_model_base as cs_model
import detect_peaks as dp
import numpy as np
import matplotlib.pyplot as plt
import place_cell_base as pl
import os
import helper_functions as hf
import scipy.stats as stats

path = os.path.abspath(os.path.dirname(pl.__file__))


mean_video_srate = 100.
total_Time = 500
_lambda = 10

x_coordinates,y_coordinates,speed,timevector = cs_model.generate_randomWalk2(input_srate = mean_video_srate,input_total_Time = total_Time,heading_srate = 10., speed_srate = 5., rho1  = 1,sigma = 0.02,mu_e  = 0.,smooth_coeff = 0.5)
All_arrival_time,I_timestamps = cs_model.generate_arrivals(_lambda,total_Time)

# modulated_timestamps = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 500)
# modulated_timestamps1 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)
# modulated_timestamps2 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 10, s = 5)
# modulated_timestamps3 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 90, s = 5)
# modulated_timestamps4 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 90, s = 5)
# modulated_timestamps5 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 10, s = 5)
# modulated_timestamps = np.unique(np.concatenate([modulated_timestamps1,modulated_timestamps2,modulated_timestamps3,modulated_timestamps4,modulated_timestamps5]))

# modulated_timestamps = np.sort(np.random.choice(modulated_timestamps,50,replace=False))
modulated_timestamps = np.sort(np.random.choice(I_timestamps,50,replace=False))

calcium_imag,timevector = cs_model.generate_CalciumSignal(modulated_timestamps,noise_level = 0.01, b = 5.)
calcium_events = modulated_timestamps.shape[0]
I_peaks = dp.detect_peaks(calcium_imag,mpd=0.5*mean_video_srate,mph=1.*np.nanstd(calcium_imag))


place_cell = pl.PlaceCell(RatSession='Sst',day = 1, ch = 1, dataset = 'Exploration',mean_video_srate = 100.,mintimespent = 0.1,
                          minvisits = 1, speed_threshold = 5, nbins_cal = 10, num_surrogates = 200,num_cores = 20, nbins_pos_x = 10, 
                          nbins_pos_y = 10, placefield_nbins_pos_x = 50, placefield_nbins_pos_y = 50)

PlaceCellObject = place_cell.run_placeMetrics(calcium_imag,timevector,x_coordinates,y_coordinates)

plt.figure()
cnt = plt.contourf(PlaceCellObject['x_center_bins'],PlaceCellObject['y_center_bins'],PlaceCellObject['place_field_smoothed'],100)
for c in cnt.collections:
    c.set_edgecolor("face")


plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title(f"MI = {PlaceCellObject['mutualInfo_zscored']:0.3f} ; Events = {calcium_events}")
plt.show()


/home/apolo/Documents/DataAnalysis/Projects/CalciumSignalAnalysis/place_cell_base.py:178: RuntimeWarning: Mean of empty slice
  calcium_mean_occupancy[yy,xx] = np.nanmean(mean_calcium_to_behavior_speed[np.logical_and(check_x_ocuppancy,check_y_ocuppancy)])


File not saved!


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [315]:
I_peaks = dp.detect_peaks(calcium_imag,mpd=1*mean_video_srate,mph=1*np.nanstd(calcium_imag))

plt.figure()
plt.plot(timevector,calcium_imag,'k')
plt.plot(timevector[I_peaks],calcium_imag[I_peaks],'r.')

plt.xlim([0,500])

plt.xticks(range(0,600,100))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [328]:

plt.figure()
cnt = plt.contourf(PlaceCellObject['x_center_bins'],PlaceCellObject['y_center_bins'],PlaceCellObject['place_field_smoothed'],100)
for c in cnt.collections:
    c.set_edgecolor("face")


plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title(f"MI = {PlaceCellObject['mutualInfo_zscored']:0.3f} ; Events = {calcium_events}")
plt.show()

os.chdir(path + '/figures/')
# plt.savefig('place_field_simulated_grid_cell.svg')
# plt.savefig('place_field_simulated_place_cell.svg')
plt.savefig('place_field_simulated_non_place_cell.svg')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [317]:
from joblib import Parallel, delayed
from sklearn.naive_bayes import GaussianNB


nbins_pos_x = 10
nbins_pos_y = 10

xbins = nbins_pos_x
ybins = nbins_pos_y

x_range = (np.nanmax(x_coordinates) - np.nanmin(x_coordinates))
x_grid_window = x_range/nbins_pos_x
x_grid = np.arange(np.nanmin(x_coordinates),np.nanmax(x_coordinates) + x_grid_window/2,x_grid_window)

y_range = (np.nanmax(y_coordinates) - np.nanmin(y_coordinates))
y_grid_window = y_range/nbins_pos_y
y_grid = np.arange(np.nanmin(y_coordinates),np.nanmax(y_coordinates) + y_grid_window/2,y_grid_window)


x_center = x_grid[0:-1] + np.diff(x_grid)/2
y_center = y_grid[0:-1] + np.diff(y_grid)/2

bin_track_vector = np.nan*np.zeros(x_coordinates.shape[0])

count = 0
for ssx in range(xbins):
    X_occup = np.logical_and(x_coordinates >= x_grid[ssx], x_coordinates < (x_grid[ssx+1]))
    for ssy in range(ybins):
        Y_occup = np.logical_and(y_coordinates >= y_grid[ssy], y_coordinates < (y_grid[ssy+1]))

        bin_track_vector[np.logical_and(X_occup,Y_occup)] = count
        count += 1

I_keep = ~np.isnan(bin_track_vector)

x_coordinates_new = x_coordinates[I_keep].copy()
y_coordinates_new = y_coordinates[I_keep].copy()

Target_Variable = bin_track_vector[I_keep].copy()

plot_window = np.arange(np.floor(-10*mean_video_srate),np.floor(+ 10*mean_video_srate)).astype(int)/mean_video_srate


Input_Variable = calcium_imag[I_keep].copy()
Input_Variable_Origianl = Input_Variable.reshape(Input_Variable.shape[0],1)   

xbins = nbins_pos_x
ybins = nbins_pos_y

x_center = x_grid[0:-1] + np.diff(x_grid)/2
y_center = y_grid[0:-1] + np.diff(y_grid)/2

bin_x_centers = np.repeat(x_center,ybins)
bin_y_centers = np.tile(y_center,xbins)

window_size = int(np.floor(Input_Variable.shape[0]/10.))
I_start = np.arange(0,Input_Variable.shape[0]+1,window_size)

results = Parallel(n_jobs=10)(delayed(parallel_input_decoding)(Input_Variable_Origianl,Target_Variable,I_start,bin_x_centers,bin_y_centers,x_coordinates_new,y_coordinates_new,fold) for fold in range(10))
continuous_error_original = np.concatenate(results)

triggered_error_original,triggered_calcium_original,smoothed_error_original = get_triggered_error(Input_Variable,continuous_error_original,mean_video_srate)
correlation_pearson = np.corrcoef(Input_Variable,smoothed_error_original)[0,1]



In [329]:

I_pre_window = np.where((plot_window>= -3) & (plot_window < -2))[0]
I_pos_window = np.where((plot_window>= 0) & (plot_window < 1))[0]

mean_triggered_error_original_baseline = np.nanmean(triggered_error_original[:,I_pre_window],1)
mean_triggered_error_original_event = np.nanmean(triggered_error_original[:,I_pos_window],1)


plt.figure()
plt.boxplot([mean_triggered_error_original_baseline,mean_triggered_error_original_event])
plt.plot([1.2,1.8],[mean_triggered_error_original_baseline,mean_triggered_error_original_event],'k-',alpha=0.5)
# plt.ylim([10,90])
plt.show()

os.chdir(path + '/figures/')
# plt.savefig('place_field_simulated_grid_cell_boxplot.svg')
# plt.savefig('place_field_simulated_place_cell_boxplot.svg')
plt.savefig('place_field_simulated_non_place_cell_boxplot.svg')

t_dependent,pval = stats.ttest_rel(mean_triggered_error_original_baseline,mean_triggered_error_original_event,alternative='greater')
effect_size = t_dependent/np.sqrt(mean_triggered_error_original_baseline.shape[0])
mean_difference = np.nanmean(mean_triggered_error_original_event) - np.nanmean(mean_triggered_error_original_baseline)
pval

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.7277650767890245

In [330]:

plt.close('all')

mean_triggered_error = np.nanmean(triggered_error_original,0)
mean_triggered_calcium = np.nanmean(triggered_calcium_original,0)

plt.figure(figsize=(18,5))
plt.plot(plot_window,triggered_error_original.T,'k',alpha=0.2)
plt.plot(plot_window,mean_triggered_error,'r',linewidth = 4)
plt.ylabel('Decoding Error (cm)',color='red',fontsize=15,fontweight='extra bold')
plt.xlabel('Time (s)',fontsize=15)
plt.grid(color='black',linestyle = '--',alpha=0.5,zorder=10)
plt.xticks(np.arange(-10,12,2))
plt.xlim([-5,5])

plt.twinx()
plt.plot(plot_window,mean_triggered_calcium,'g',linewidth = 4)
plt.ylabel('Calcium Signal',color='green',fontsize=15,fontweight='extra bold')
plt.title('Simulated Cell')
plt.show()

os.chdir(path + '/figures/')
# plt.savefig('place_field_simulated_grid_cell_decoding.svg')
# plt.savefig('place_field_simulated_place_cell_decoding.svg')
plt.savefig('place_field_simulated_non_place_cell_decoding.svg')




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [321]:

import matplotlib.gridspec as gridspec
    
plt.close('all')

mean_triggered_error = np.nanmean(triggered_error_original,0)
mean_triggered_calcium = np.nanmean(triggered_calcium_original,0)

plt.figure(figsize=(18,5))
gridspec.GridSpec(1,3)

plt.subplot2grid((1,3), (0,0), colspan=2, rowspan=1)
#     plt.subplot(1,2,1)
plt.plot(plot_window,triggered_error_original.T,'k',alpha=0.2)
plt.plot(plot_window,mean_triggered_error,'r',linewidth = 4)
plt.ylabel('Decoding Error (cm)',color='red',fontsize=15,fontweight='extra bold')
plt.xlabel('Time (s)',fontsize=15)
plt.grid(color='black',linestyle = '--',alpha=0.5,zorder=10)
plt.xticks(np.arange(-10,12,2))
plt.xlim([-5,5])
# plt.ylim([0,26])

plt.twinx()
plt.plot(plot_window,mean_triggered_calcium,'g',linewidth = 4)
plt.ylabel('Calcium Signal',color='green',fontsize=15,fontweight='extra bold')
plt.title('Simulated Cell')

plt.subplot2grid((1,3), (0,2))
#     plt.subplot(1,2,2)
plt.pcolormesh(PlaceCellObject['x_grid'],PlaceCellObject['y_grid'],PlaceCellObject['signalMap'])
plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
# plt.clim([0,2])
#         plt.axis('scaled')
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [322]:
all_dist_fields = []
for _ in range(1000):
    rand_pt1 = np.random.choice(PlaceCellObject['events_index'],1)
    rand_pt2 = np.random.choice(PlaceCellObject['events_index'],1)

    dist = np.sqrt((x_coordinates[rand_pt1] - x_coordinates[rand_pt2])**2 + (y_coordinates[rand_pt1]-y_coordinates[rand_pt2])**2)
    all_dist_fields.append(dist)
np.nanmean(all_dist_fields)

54.33744301954309

In [323]:
all_dist = []
for _ in range(1000):
    rand_pt1 = np.random.choice(range(x_coordinates.shape[0]),1)
    rand_pt2 = np.random.choice(range(x_coordinates.shape[0]),1)

    dist = np.sqrt((x_coordinates[rand_pt1] - x_coordinates[rand_pt2])**2 + (y_coordinates[rand_pt1]-y_coordinates[rand_pt2])**2)[0]
    all_dist.append(dist)
np.nanmean(all_dist)

49.650917180114995

In [310]:
def generate_surrogate(signal_in,permi):
    signal_in_shuffled = []
    I_break = np.random.choice(np.arange(int(signal_in.shape[0]*0.1),int(signal_in.shape[0]*0.9)),1)[0].astype(int)

    if np.mod(permi,4) == 0:
        signal_in_shuffled = np.concatenate([signal_in[I_break:], signal_in[0:I_break]])
    elif np.mod(permi,4) == 1:
        signal_in_shuffled = np.concatenate([signal_in[:I_break:-1], signal_in[0:I_break+1]])
    elif np.mod(permi,4) == 2:
        signal_in_shuffled = np.concatenate([signal_in[I_break:], signal_in[I_break-1::-1]])
    else:   
        signal_in_shuffled = np.concatenate([signal_in[I_break:], signal_in[0:I_break]])
        signal_in_shuffled = signal_in_shuffled[::-1]
    
    
    return signal_in_shuffled


def get_triggered_error(signal_in,nearest_dist_to_predicted_bin,mean_video_srate):
    
    smoothed_error = hf.smooth(nearest_dist_to_predicted_bin,window_len=30)
    I_peaks = dp.detect_peaks(signal_in,mpd=1*mean_video_srate,mph=1*np.nanstd(signal_in))

    triggered_error = []
    triggered_calcium = []
    for ii in I_peaks:
        if ((ii - 10*mean_video_srate) > 0) and (ii + 10*mean_video_srate < smoothed_error.shape[0]):
            error_window = np.arange(np.floor(ii - 10*mean_video_srate),np.floor(ii + 10*mean_video_srate)).astype(int)
            triggered_calcium.append(signal_in[error_window])
            triggered_error.append(smoothed_error[error_window])

    triggered_error = np.array(triggered_error)
    triggered_calcium = np.array(triggered_calcium)
    return triggered_error,triggered_calcium,smoothed_error

def parallel_input_decoding(Input_Variable_Cell,Target_Variable,I_start,bin_x_centers,bin_y_centers,x_coordinates_new,y_coordinates_new,fold):


        if fold == 9:
            Trials_testing_set =  np.arange(I_start[fold],Input_Variable_Cell.shape[0]).astype(int)
        else:
            Trials_testing_set =  np.arange(I_start[fold],I_start[fold+1]).astype(int)

        Trials_training_set = np.setdiff1d(range(Input_Variable_Cell.shape[0]),Trials_testing_set)

        X_train = Input_Variable_Cell[Trials_training_set,:].copy()
        y_train = Target_Variable[Trials_training_set].copy()

        X_test = Input_Variable_Cell[Trials_testing_set,:].copy()
        y_test = Target_Variable[Trials_testing_set].copy()


        priors_in = np.ones(np.unique(Target_Variable).shape[0])/np.unique(Target_Variable).shape[0]

        gnb = GaussianNB(priors = priors_in)
        gnb.fit(X_train, y_train)

        accuracy_original = gnb.score(X_test, y_test)


        y_pred = gnb.predict(X_test)
        y_test = y_test.astype(int)
        y_pred = y_pred.astype(int)

        diffx = (bin_x_centers[y_pred]-x_coordinates_new[Trials_testing_set])**2
        diffy = (bin_y_centers[y_pred]-y_coordinates_new[Trials_testing_set])**2

        nearest_dist_to_predicted_bin  = np.sqrt(diffx + diffy)


        return nearest_dist_to_predicted_bin

In [309]:
np.unique(Target_Variable).shape


(100,)